# Install and Download Dependencies

In [23]:
!pip install spacy scikit-learn requests beautifulsoup4 matplotlib langdetect

# Download SpaCy language models for French and English
!python -m spacy download fr_core_news_sm
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 9.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 24.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Import Libraries and Configure Settings

In [ ]:
import os
import pickle
import re
from concurrent.futures import ThreadPoolExecutor

import matplotlib.pyplot as plt
import requests
import spacy
from bs4 import BeautifulSoup
from langdetect import LangDetectException, detect
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import (CountVectorizer, TfidfVectorizer,
                                             ENGLISH_STOP_WORDS)
from sklearn.metrics.pairwise import cosine_similarity
from spacy.lang.fr.stop_words import STOP_WORDS as FRENCH_STOP_WORDS

# Load NLP models
nlp_fr = spacy.load("fr_core_news_sm")
nlp_en = spacy.load("en_core_web_sm")

# URLs to index initially
target_urls = [
    "https://fr.wikipedia.org/wiki/Intelligence_artificielle",
    "https://fr.wikipedia.org/wiki/Apprentissage_automatique",
    "https://fr.wikipedia.org/wiki/Apprentissage_supervisé",
    "https://fr.wikipedia.org/wiki/Apprentissage_profond",
    "https://fr.wikipedia.org/wiki/Cybersécurité",
    "https://fr.wikipedia.org/wiki/Cybercriminalité",
    "https://fr.wikipedia.org/wiki/Système_d'exploitation",
    "https://fr.wikipedia.org/wiki/Réseau_informatique",
    "https://fr.wikipedia.org/wiki/Programmation_orientée_objet",
    "https://fr.wikipedia.org/wiki/Base_de_données",
    "https://fr.wikipedia.org/wiki/Cloud_computing",
    "https://fr.wikipedia.org/wiki/Big_data",
    "https://fr.wikipedia.org/wiki/Algorithme"
]


# Define Data Fetching and Processing Functions

In [ ]:
def get_text_from_url(url):
    """Récupère et extrait le texte brut d'une URL."""
    try:
        response = requests.get(url, timeout=15, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p')
        return ' '.join([para.get_text() for para in paragraphs])
    except requests.RequestException as e:
        print(f"Erreur lors de la récupération de l'URL {url}: {e}")
        return ""

def fetch_all_texts(urls):
    """Télécharge le texte d'une liste d'URLs en parallèle."""
    with ThreadPoolExecutor(max_workers=5) as executor:
        return list(executor.map(get_text_from_url, urls))

def detect_language(text):
    """Détecte la langue d'un extrait de texte donné."""
    try:
        return detect(text[:500])
    except LangDetectException:
        return 'unknown'

def clean_and_lemmatize_text(text):
    """Nettoie et lemmatise le texte en utilisant le modèle de langue approprié."""
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^a-zA-ZÀ-ÿ\s]', '', text)
    lang = detect_language(text)
    nlp = nlp_fr if lang == 'fr' else nlp_en
    doc = nlp(text.lower())
    return ' '.join([
        token.lemma_ for token in doc
        if token.text not in combined_stop_words and not token.is_punct and not token.is_space
    ])

# Fonctions de mise en cache
def save_cache(data, filename):
    with open(filename, "wb") as f:
        pickle.dump(data, f)

def load_cache(filename):
    if os.path.exists(filename):
        with open(filename, "rb") as f:
            return pickle.load(f)
    return None

print("✅ Les fonctions utilitaires pour le traitement des données et la mise en cache sont définies.")

# Define Model and Search Functions

In [ ]:
def build_tfidf_model(docs):
    """Construit et retourne un vectoriseur TF-IDF et la matrice correspondante."""
    vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, ngram_range=(1, 2))
    matrix = vectorizer.fit_transform(docs)
    return vectorizer, matrix

def build_lda_model(docs, n_topics=4):
    """Construit un modèle LDA et son vectoriseur correspondant."""
    # Convertir le frozenset en liste pour le paramètre stop_words
    count_vectorizer = CountVectorizer(max_df=0.90, min_df=2, stop_words=list(combined_stop_words))
    count_matrix = count_vectorizer.fit_transform(docs)
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda.fit(count_matrix)
    return lda, count_vectorizer

def search_tfidf(query, vectorizer, matrix, urls, docs, top_n=10):
    """Effectue une recherche en utilisant un modèle TF-IDF pré-calculé."""
    cleaned_query = clean_and_lemmatize_text(query)
    query_vec = vectorizer.transform([cleaned_query])
    cosine_sim = cosine_similarity(query_vec, matrix).flatten()
    ranked_indices = cosine_sim.argsort()[::-1]
    results = []
    for i in ranked_indices:
        if cosine_sim[i] > 0 and len(results) < top_n:
            results.append((urls[i], docs[i][:250], cosine_sim[i]))
    return results

print("✅ Les fonctions pour construire les modèles et effectuer la recherche sont définies.")

# Define Visualization Functions

In [ ]:
def plot_tfidf_scores(results):
    """Visualise les résultats de la recherche TF-IDF sous forme de diagramme à barres horizontales."""
    if not results:
        print("Aucun résultat pertinent à afficher.")
        return
    urls = [os.path.basename(result[0]) for result in results]
    scores = [result[2] for result in results]
    plt.figure(figsize=(12, 8))
    plt.barh(urls, scores, color='skyblue')
    plt.xlabel('Score de similarité')
    plt.ylabel('Document')
    plt.title('Scores de similarité TF-IDF pour la requête')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

def plot_lda_topics(model, vectorizer, n_top_words=10):
    """Visualise les mots les plus importants pour chaque topic du modèle LDA."""
    feature_names = vectorizer.get_feature_names_out()
    fig, axes = plt.subplots(model.n_components, 1, figsize=(10, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[-n_top_words:]
        top_features = feature_names[top_features_ind]
        weights = topic[top_features_ind]
        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Sujet #{topic_idx + 1}", fontdict={"fontsize": 14})
        ax.invert_yaxis()
        ax.tick_params(axis='both', which='major', labelsize=12)
    plt.suptitle("Mots principaux par topic (LDA)", fontsize=18)
    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
    plt.show()

print("✅ Les fonctions de visualisation sont définies.")

# Run the Main Program

In [ ]:
# --- Chargement et Indexation des Données ---
CACHE_FILE = "cached_data.pkl"
cached_data = load_cache(CACHE_FILE)

if cached_data:
    print("Chargement des données pré-traitées depuis le cache...")
    documents, urls = cached_data
else:
    print("Récupération et traitement des documents depuis le web...")
    raw_documents = fetch_all_texts(target_urls)
    successful_urls = [url for i, url in enumerate(target_urls) if raw_documents[i]]
    raw_documents = [doc for doc in raw_documents if doc]

    # Définir les mots vides (stop words) combinés
    combined_stop_words = ENGLISH_STOP_WORDS.union(FRENCH_STOP_WORDS)

    print("Nettoyage et lemmatisation des textes (cela peut prendre un moment)...")
    documents = [clean_and_lemmatize_text(doc) for doc in raw_documents]

    save_cache((documents, successful_urls), CACHE_FILE)
    urls = successful_urls

print(f"\n{len(documents)} documents indexés avec succès.")

# --- Entraînement des Modèles ---
print("Construction des modèles TF-IDF et LDA...")
tfidf_vectorizer, tfidf_matrix = build_tfidf_model(documents)
lda_model, lda_vectorizer = build_lda_model(documents, n_topics=4)
print("Les modèles sont prêts. 🚀")

# --- Boucle Interactive de Recherche et d'Analyse ---
while True:
    print("\n" + "="*50)
    query = input("Entrez votre requête (ou 'exit' pour quitter) : ")
    if query.lower() == "exit":
        break

    # 1. Effectuer la recherche TF-IDF
    print("\n--- Résultats de la recherche TF-IDF ---")
    tfidf_results = search_tfidf(query, tfidf_vectorizer, tfidf_matrix, urls, documents)

    if not tfidf_results:
        print("Aucun document correspondant à votre requête n'a été trouvé.")
        continue

    for url, snippet, score in tfidf_results:
        print(f"URL : {url} (Score : {score:.4f})")
        print(f"   Extrait : {snippet}...\n")

    # 2. Visualiser les résultats TF-IDF
    print("\n--- Visualisation des scores TF-IDF ---")
    plot_tfidf_scores(tfidf_results)

    # 3. Afficher et visualiser les sujets LDA
    print("\n--- Résultats de la modélisation de topic (LDA) ---")
    print("Voici les principaux topic trouvés dans l'ensemble des documents indexés.")
    plot_lda_topics(lda_model, lda_vectorizer)

    # 4. Analyse Comparative
    print("\n---  Analyse Comparative ---")
    print(f"La recherche TF-IDF a trouvé des documents spécifiquement pertinents pour votre requête : '{query}'.")
    print("Le modèle LDA montre les thèmes généraux présents dans l'ensemble de la collection de documents.")
    print("En les comparant, vous pouvez voir si le document le plus pertinent pour votre requête correspond à l'un des topic principaux.")